# Classification



In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import datasets

from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split

In [ ]:
higgs = datasets.fetch_openml(name='higgs', version=2)

In [ ]:
df = pd.DataFrame(higgs.data, columns=higgs.feature_names)

In [ ]:
df.head()

In [ ]:
df.iloc[-1]

In [ ]:
df.columns

In [ ]:
X = df.loc[:, ['m_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']].iloc[:-1].values

y = higgs.target[:-1].astype(np.int)

In [ ]:
y

Binnary Classification: Two Classes (0 and 1)

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)

In [ ]:
GaussianNB?

In [ ]:
GNB = GaussianNB()

In [ ]:
GNB.fit(X_train, y_train)

In [ ]:
GNB.score(X_train, y_train)

In [ ]:
GNB.score(X_test, y_test)

In [ ]:
help(GNB.score)

In [ ]:
y_pred = GNB.predict(X_test)

y_pred

In [ ]:
y_test

## Confusion Matrix

Definition:

$$\mathrm{ CM = \left(\begin{matrix}
 \mathrm{tn} & \mathrm{fp} \\ 
 \mathrm{fn} & \mathrm{tp}
\end{matrix}\right) }$$

Where

    tp: True Positive values

    fp: False Positive values

    tn: True Negative values

    fn: False Negative values

In [ ]:
y_pred == 0

In [ ]:
p = y_pred == 1 # positives
n = y_pred == 0 # negatives

In [ ]:
rp = y_test == 1 # real positives
rn = y_test == 0 # real negatives

In [ ]:
tp = (p & rp).sum() # true positives
tn = (n & rn).sum() # true negatives
fp = (p & rn).sum() # false positives
fn = (n & rp).sum() # false negatives

In [ ]:
tp, tn, fp, fn

In [ ]:
(tp + tn + fp + fn) == len(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

cm

In [ ]:
tp == cm[1, 1], tn == cm[0, 0], fp == cm[0, 1], fn == cm[1, 0]

In [ ]:
y_test

In [ ]:
y_proba = GNB.predict_proba(X_test)

y_proba

In [ ]:
y_proba = y_proba[:, 1]

y_proba

In [ ]:
np.unique?

In [ ]:
probas = np.sort(y_proba)

In [ ]:
probas

In [ ]:
probas.shape

In [ ]:
probas = probas[::10]

probas.shape

In [ ]:
cms = np.stack([
    confusion_matrix(y_test, y_proba > proba)
    for proba in probas
])

In [ ]:
tps = cms[:, 1, 1]
tns = cms[:, 0, 0]
fps = cms[:, 0, 1]
fns = cms[:, 1, 0]

In [ ]:
plt.plot(probas, tps, label = 'True Positive')
plt.plot(probas, fps, label = 'False Positive')
plt.plot(probas, tns, label = 'True Negative')
plt.plot(probas, fns, label = 'False Negative')

plt.legend()

plt.xlabel('Treshold')

Acurácia:

$$ \mathrm{Accuracy = \frac{tp+tn}{tp+tn+fp+fn}} $$

In [ ]:
acc = (tps + tns)/(tps + tns + fps + fns)

In [ ]:
plt.plot(probas, acc)

In [ ]:
acc.max(), probas[acc.argmax()]

## Receiver Operating Characteristic Curve (ROC curve)

True Positive Rate:

$$ \mathrm{TPR = \frac{tp}{tp + fn}} $$

False Positive Rate:

$$ \mathrm{FPR = \frac{fp}{fp + tn}} $$

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, treshold = roc_curve(y_test, y_proba)

In [ ]:
plt.plot(fpr, tpr, label='GNB')

plt.xlabel('FPR')
plt.ylabel('TPR')

plt.legend()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
KNR = KNeighborsClassifier(n_neighbors=5)

In [ ]:
KNR.fit(X_train, y_train)

In [ ]:
y_proba_knr = KNR.predict_proba(X_test)

y_proba_knr = y_proba_knr[:, 1]

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_proba_knr)

plt.plot(fpr, tpr, label='KNR')

fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.plot(fpr, tpr, label='GNB')

plt.xlabel('FPR')
plt.ylabel('TPR')

plt.legend()

Score: Area Under the ROC Curve (AUC)

Perfect Classifier: AUC = 1

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_proba), roc_auc_score(y_test, y_proba_knr)

## Precision-Recall Curve

Precision:

$$ \mathcal{P} = \mathrm{\frac{tp}{tp + fp}} $$

Recall (True Positive Rate):

$$ \mathcal{R} = \mathrm{\frac{tp}{tp + fn}} $$

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
y, x, _ = precision_recall_curve(y_test, y_proba_knr)

plt.plot(x, y, label='KNR')

y, x, _ = precision_recall_curve(y_test, y_proba)

plt.plot(x, y, label='GNB')

plt.xlabel('Recall')
plt.ylabel('Precision')

plt.legend()

Score: Average Precision(~Area Under the Precision-Recall Curve)

$$ \text{AP} = \sum_n (\mathcal{R}_n - \mathcal{R}_{n-1}) \mathcal{P}_n $$

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
average_precision_score(y_test, y_proba), average_precision_score(y_test, y_proba_knr)